In [9]:
%load_ext autoreload
%autoreload 2

from opengsync_db import categories, DBHandler
from opengsync_db.ext.autosession import set_db
%load_ext opengsync_db.ext.autosession

from IPython.display import display
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The opengsync_db.ext.autosession extension is already loaded. To reload it, use:
  %reload_ext opengsync_db.ext.autosession


True

In [10]:
db = DBHandler()
db.connect(
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_SERVER_IP"],
    port=os.environ["POSTGRES_PORT"],
    db=os.environ["POSTGRES_DB"],
)
set_db(db)

LOG: Connected to DB 'postgresql+psycopg://bsf-opengsync.int.cemm.at:5432/opengsync_db'


### 0. Categories (Enums)

In [ ]:
# These are used to store statuses or types, e.g. LibraryType, ExperimentStatus, GenomeRef
# Each enum has a unique id inside it's own enum-class
# these are defined in services/opengsync-app/opengsync-db/opengsync_db/categories/
categories.LibraryType.BULK_RNA_SEQ.id, categories.LibraryType.BULK_RNA_SEQ.name, categories.GenomeRef.COVID.organism_tax_id

(101, 'Poly-A RNA-Seq', 2697049)

### 1. Get All Libraries from Sequencing Run (Before demultiplexing flowcell)

In [ ]:
experiment = db.experiments["BSF_1776"]
print(experiment)
print(experiment.flowcell_type)
print(experiment.operator)
print(experiment.num_lanes)
print(experiment.status)

In [ ]:
# This dataframe should be sufficient to demultiplex a flowcell
experiment_libraries = db.pd.get_flowcell(experiment_id=experiment.id)
experiment_libraries.sample(10)

#### 1.1 Grouping by Sequencing Request

In [ ]:
# Demultiplexed fastq-files should be grouped by 'seq_request_id' as each Sequencing Request can have only one Requestor
for (seq_request_id), df in experiment_libraries.groupby("seq_request_id"):
    print(f"seq_request_id: {seq_request_id}")
    display(df.head(20))
    print("...\n")


#### 1.2 Filtering Library Type

In [ ]:
experiment_libraries[experiment_libraries["library_type"] == categories.LibraryType.WGS].head(3)

### 2. Downstream Projects (after demultiplexing)

In [34]:
project = db.projects["BSA_1041"]
# or if identifier not assigned you can also use ID (integer)
# project = db.projects[61]
project

Project(id: 61, identifier: BSA_1041)

In [ ]:
db.pd.get_project_samples(project.id)

In [ ]:
# this dataframe should be enough to find all fastq-files generated from demultiplexing step
# projects can contain multiple requests, and libraries from multiple experiments
project_libraries = db.pd.get_project_libraries(project.id)
project_libraries

In [ ]:
# Sequencing Requests associated with a project
db.pd.get_project_seq_requests(project.id)

### 3. Using ORM instead of Pandas Dataframe

In [ ]:
seq_request = db.seq_requests[1]
print(seq_request.contact_person)
print(seq_request.comments)
print(seq_request.data_delivery_mode)

### 4. OpeNGSync API

In [ ]:
# pip install -e ./services/opengsync-app/opengsync-api/

from opengsync_api import OpeNGSyncAPI
api = OpeNGSyncAPI("https://domain.com", os.environ["OPENGSYNC_API_KEY"])
api.authenticate()

#### 4.1 Set library lane reads (after demultiplexing)

In [ ]:
api.set_library_lane_reads(
    library_id=1,  # or None for Undetermined reads,
    experiment_name="BSF_1776",  # experiment must exist
    lane=1,  # lane must exist for the experiment
    num_reads=1000000,  # number of reads assigned to this library in this lane
    qc=dict(r1_mean_quality=35, r2_mean_quality=34) # free-form dict for QC metrics
)

In [ ]:
db.pd.get_experiment_stats(experiment_id=experiment.id)

#### 4.3 Release Data (Processed and Raw data together)

In [ ]:
# all sub-directories will be shared
api.add_data_path(
    project_id=project.id,
    path="/data/path/to/processed/data/processed_project_1",
    path_type=categories.DataPathType.DIRECTORY
)
api.add_data_path(
    project_id=project.id,
    path="/data/path/to/raw/data/raw_data_for_project_1",
    path_type=categories.DataPathType.DIRECTORY
)

In [ ]:
# finds all emails that are specified in the latest sequencing request of the project
share_email_df = db.pd.get_project_latest_request_share_emails(project.id)
recipients = share_email_df["email"].tolist()

In [ ]:
# sends email to recipients
api.release_project_data(
    project_id=project.id,
    internal_access=True,  # additional instructions for internal users
    recipients=recipients,
    anonymous_send=False,
    time_valid_min=60*24*14,  # 2 weeks
)